In [ ]:
import os
import pandas as pd
import json
import folium
import copy

#### 1. Europe

In [ ]:
# load the data regarding the ratio of unemployement
europe_unemploy_rate_path = r'topojson/lfsa_urgan_1_Data.csv'
europe_unemploy_rate = pd.read_csv(europe_unemploy_rate_path)
europe_unemploy_rate = europe_unemploy_rate.loc[\
    (europe_unemploy_rate.TIME==2016) & (europe_unemploy_rate.SEX=="Total") & (~europe_unemploy_rate.GEO.str.contains("European|Euro")), \
    ["GEO","Value"]]
europe_unemploy_rate.loc[europe_unemploy_rate.GEO.str.contains("Germany"), "GEO"] = "Germany"
europe_unemploy_rate.loc[europe_unemploy_rate.GEO.str.contains("Former Yugoslav Republic of Macedonia"), "GEO"] = "The former Yugoslav Republic of Macedonia"
print("We have the unemployement rate the following states: ", europe_unemploy_rate.shape[0])
europe_unemploy_rate

In [ ]:
# load the europe data (geometry of the states) 
europe_topo_path = r'topojson/europe.topojson.json'
topo_data = json.load(open(europe_topo_path))
topo_states = [state["properties"]["NAME"] for state in topo_data["objects"]["europe"]["geometries"]]
print("We draw the boundaries of the following states:")
len(topo_states), topo_states

In [ ]:
# we don't have the data of all the states!
missing_states = list(set(topo_states)-set(europe_unemploy_rate.GEO))
print("We are missing the unemployement rate about these states:")
len(missing_states), missing_states

In [ ]:
def fill_invalid(state):
    if state in list(missing_states):
        return 0.7
    else:
        return 0

In [ ]:
topo_data['objects']['europe']["geometries"]

In [ ]:
europe_location = [55, 15]

m = folium.Map(location=europe_location, zoom_start=3)

# fill the states with colors depending on their unemployement rate
m.choropleth(
    geo_data=copy.deepcopy(topo_data), 
    topojson='objects.europe',
    data=europe_unemploy_rate,
    columns=['GEO', 'Value'],
    #threshold_scale=[0, 5, 10, 15, 20, 25],
    key_on='properties.NAME',
    fill_color='BuGn', 
    fill_opacity=0.7, 
#     line_opacity=1,
    legend_name='Percentage of unemployement (%)')

# draw better boundaries
folium.TopoJson(
    topo_data,
    'objects.europe',
    name='topojson',
    style_function=lambda geometry: {
        # boundaries
        'color' : 'black',
        'weight' : 1,
        # fill invalid states with grey
        'fillColor': "#424949", 
        'fillOpacity': fill_invalid(geometry["properties"]["NAME"]), 
        }
).add_to(m)

m

#### 2. Switzerland

In [ ]:
# load the data regarding the ratio of unemployement
ch_unemploy_rate_path = r'topojson/ch_unemploy_rate.csv'
ch_unemploy_rate = pd.read_csv(ch_unemploy_rate_path)
ch_unemploy_rate = ch_unemploy_rate.loc[
    ch_unemploy_rate.Cantone != "Totale",
    ["Cantone", "Settembre 2017"]
]
ch_unemploy_rate = ch_unemploy_rate.rename(columns={'Settembre 2017': 'Value'})
print("We have the unemployement rate the following cantons: ", ch_unemploy_rate.shape[0])
ch_unemploy_rate

In [ ]:
# load the swiss data (geometry of the cantons) 
ch_topo_path = r'topojson/ch-cantons.topojson.json'
topo_data_ch = json.load(open(ch_topo_path))
topo_cantons = [state["properties"]["name"] for state in topo_data_ch["objects"]["cantons"]["geometries"]]
print("We draw the boundaries of the following cantons:")
len(topo_cantons), topo_cantons

In [ ]:
topo_data_ch['objects']['cantons']["geometries"][0]["properties"]["name"]

In [ ]:
ch_location = [46.8, 8.5]
m = folium.Map(location=ch_location, zoom_start=8)

# fill the cantons with colors depending on their unemployement rate
m.choropleth(
    geo_data=copy.deepcopy(topo_data_ch), 
    topojson='objects.cantons',
    data=ch_unemploy_rate,
    columns=['Cantone', 'Value'],
    #threshold_scale=[0, 5, 10, 15, 20, 25],
    key_on='properties.name',
    fill_color='BuGn', 
    fill_opacity=0.7, 
#     line_opacity=1,
    legend_name='Percentage of unemployement (%)')
m